In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Create a sigmoid layer:
layers.Dense(64, activation='sigmoid')
# Or:
layers.Dense(64, activation=tf.sigmoid)

# A linear layer with L1 regularization of factor 0.01 applied to the kernel matrix:
layers.Dense(64, kernel_regularizer=keras.regularizers.l1(0.01))
# A linear layer with L2 regularization of factor 0.01 applied to the bias vector:
layers.Dense(64, bias_regularizer=keras.regularizers.l2(0.01))

# A linear layer with a kernel initialized to a random orthogonal matrix:
layers.Dense(64, kernel_initializer='orthogonal')
# A linear layer with a bias vector initialized to 2.0s:
layers.Dense(64, bias_initializer=keras.initializers.constant(2.0))

In [2]:
model = keras.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(keras.layers.Dense(64, activation='relu'))
# Add another:
model.add(keras.layers.Dense(64, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(keras.layers.Dense(10, activation='softmax'))

tf.keras.Model.compile takes three important arguments:

* optimizer: This object specifies the training procedure. Pass it optimizer instances from the tf.train module, such as AdamOptimizer, RMSPropOptimizer, or GradientDescentOptimizer.
* loss: The function to minimize during optimization. Common choices include mean square error (mse), categorical_crossentropy, and binary_crossentropy. Loss functions are specified by name or by passing a callable object from the tf.keras.losses module.
* metrics: Used to monitor training. These are string names or callables from the tf.keras.metrics module.

In [9]:
# Configure a model for categorical classification.
model.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
              loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])

In [10]:
data = np.random.random((1000, 32))
labels = np.random.randint(low = 1, high=10, size=(1000, 10))

val_data = np.random.random((100, 32))
val_labels = np.random.randint(low = 1, high=10, size=(100, 10))

# model.fit(data, labels, epochs=10, batch_size=32,
#           validation_data=(val_data, val_labels))

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32).repeat()

val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))
val_dataset = val_dataset.batch(32).repeat()

model.fit(dataset, epochs=10, steps_per_epoch=30,
          validation_data=val_dataset,
          validation_steps=3)

Epoch 1/10
30/30 [==============================] - 0s 11ms/step - loss: 114.6922 - categorical_accuracy: 0.1010 - val_loss: 116.2324 - val_categorical_accuracy: 0.1354
Epoch 2/10
30/30 [==============================] - 0s 1ms/step - loss: 114.6473 - categorical_accuracy: 0.1083 - val_loss: 116.0136 - val_categorical_accuracy: 0.0521
Epoch 3/10
30/30 [==============================] - 0s 1ms/step - loss: 114.4130 - categorical_accuracy: 0.1094 - val_loss: 113.1709 - val_categorical_accuracy: 0.1042
Epoch 4/10
30/30 [==============================] - 0s 1ms/step - loss: 114.3887 - categorical_accuracy: 0.1240 - val_loss: 117.0150 - val_categorical_accuracy: 0.0938
Epoch 5/10
30/30 [==============================] - 0s 1ms/step - loss: 114.4329 - categorical_accuracy: 0.1156 - val_loss: 116.2336 - val_categorical_accuracy: 0.1146
Epoch 6/10
30/30 [==============================] - 0s 1ms/step - loss: 114.1415 - categorical_accuracy: 0.1115 - val_loss: 115.9885 - val_categorical_accuracy

In [12]:
model.evaluate(dataset, steps=30)

30/30 [==============================] - 0s 2ms/step


[114.34282353719075, 0.13229166666666667]

## Funtional API

In [13]:
'''
Building a model with the functional API works like this:

1. A layer instance is callable and returns a tensor.
2. Input tensors and output tensors are used to define a tf.keras.Model instance.
3. This model is trained just like the Sequential model.

'''

'\nBuilding a model with the functional API works like this:\n\n1. A layer instance is callable and returns a tensor.\n2. Input tensors and output tensors are used to define a tf.keras.Model instance.\n3. This model is trained just like the Sequential model.\n\n'

In [14]:
inputs = keras.Input(shape=(32,))  # Returns a placeholder tensor

# A layer instance is callable on a tensor, and returns a tensor.
x = keras.layers.Dense(64, activation='relu')(inputs)
x = keras.layers.Dense(64, activation='relu')(x)
predictions = keras.layers.Dense(10, activation='softmax')(x)

# Instantiate the model given inputs and outputs.
model = keras.Model(inputs=inputs, outputs=predictions)

# The compile step specifies the training configuration.
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Trains for 5 epochs
model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 247us/step - loss: 115.0030 - acc: 0.1030
Epoch 2/5
1000/1000 [==============================] - 0s 35us/step - loss: 114.6100 - acc: 0.1030
Epoch 3/5
1000/1000 [==============================] - 0s 47us/step - loss: 114.5181 - acc: 0.1160
Epoch 4/5
1000/1000 [==============================] - 0s 44us/step - loss: 114.4268 - acc: 0.1200
Epoch 5/5
1000/1000 [==============================] - 0s 50us/step - loss: 114.3877 - acc: 0.1310


## Customize Model and Layer

In [15]:
class MyModel(keras.Model):

    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # Define your layers here.
        self.dense_1 = keras.layers.Dense(32, activation='relu')
        self.dense_2 = keras.layers.Dense(num_classes, activation='sigmoid')

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        x = self.dense_1(inputs)
        return self.dense_2(x)

    def compute_output_shape(self, input_shape):
        # You need to override this function if you want to use the subclassed model
        # as part of a functional-style model.
        # Otherwise, this method is optional.
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)


# Instantiates the subclassed model.
model = MyModel(num_classes=10)

# The compile step specifies the training configuration.
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Trains for 5 epochs.
model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 225us/step - loss: 115.2690 - acc: 0.1130
Epoch 2/5
1000/1000 [==============================] - 0s 31us/step - loss: 114.6835 - acc: 0.1150
Epoch 3/5
1000/1000 [==============================] - 0s 34us/step - loss: 114.5927 - acc: 0.1120
Epoch 4/5
1000/1000 [==============================] - 0s 36us/step - loss: 114.5263 - acc: 0.1020
Epoch 5/5
1000/1000 [==============================] - 0s 33us/step - loss: 114.4817 - acc: 0.1110


In [17]:
class MyLayer(keras.layers.Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel',
                                      shape=shape,
                                      initializer='uniform',
                                      trainable=True)
        # Be sure to call this at the end
        super(MyLayer, self).build(input_shape)

    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)

    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)


# Create a model using the custom layer
model = keras.Sequential([MyLayer(10),
                          keras.layers.Activation('softmax')])

# The compile step specifies the training configuration
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Trains for 5 epochs.
model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 193us/step - loss: 126.8104 - acc: 0.1150
Epoch 2/5
1000/1000 [==============================] - 0s 27us/step - loss: 121.0607 - acc: 0.1040
Epoch 3/5
1000/1000 [==============================] - 0s 29us/step - loss: 118.9924 - acc: 0.0950
Epoch 4/5
1000/1000 [==============================] - 0s 30us/step - loss: 118.2887 - acc: 0.0960
Epoch 5/5
1000/1000 [==============================] - 0s 30us/step - loss: 118.0103 - acc: 0.0880


## Callbacks
A callback is an object passed to a model to customize and extend its behavior during training. You can write your own custom callback, or use the built-in tf.keras.callbacks that include:

* tf.keras.callbacks.ModelCheckpoint: Save checkpoints of your model at regular intervals.
* tf.keras.callbacks.LearningRateScheduler: Dynamically change the learning rate.
* tf.keras.callbacks.EarlyStopping: Interrupt training when validation performance has stopped improving.
* tf.keras.callbacks.TensorBoard: Monitor the model's behavior using TensorBoard.

In [ ]:
callbacks = [
  # Interrupt training if `val_loss` stops improving for over 2 epochs
  keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
  # Write TensorBoard logs to `./logs` directory
  keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(data, labels, batch_size=32, epochs=5, callbacks=callbacks,
          validation_data=(val_data, val_targets))

## Distribution
A tf.keras.Model can be trained with the tf.estimator API by converting the model to an tf.estimator.Estimator object with tf.keras.estimator.model_to_estimator.

In [39]:
model = keras.Sequential()
model.add(keras.layers.Dense(16, activation='relu', input_shape=(10,)))
model.add(keras.layers.Dense(1, activation='sigmoid'))

optimizer = tf.train.GradientDescentOptimizer(0.2)

model.compile(loss='binary_crossentropy', optimizer=optimizer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 16)                176       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [40]:
def input_fn():
    x = np.random.random((1024, 10))
    y = np.random.randint(2, size=(1024, 1))
    x = tf.cast(x, tf.float32)
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.repeat(10)
    dataset = dataset.batch(32)
    return dataset

In [41]:
# strategy = tf.contrib.distribute.MirroredStrategy()
# config = tf.estimator.RunConfig(train_distribute=strategy)

In [47]:
import shutil
shutil.rmtree('/tmp/model_dir')

In [48]:
keras_estimator = keras.estimator.model_to_estimator(
  keras_model=model,
#   config=config,
  model_dir='/tmp/model_dir')

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_service': None, '_log_step_count_steps': 100, '_train_distribute': None, '_save_checkpoints_steps': None, '_is_chief': True, '_global_id_in_cluster': 0, '_evaluation_master': '', '_task_id': 0, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_task_type': 'worker', '_num_worker_replicas': 1, '_device_fn': None, '_session_config': None, '_model_dir': '/tmp/model_dir', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x128139ef0>, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_num_ps_replicas': 0}


In [49]:
keras_estimator.train(input_fn=input_fn, steps=10)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/model_dir/keras_model.ckpt
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/model_dir/model.ckpt.
INFO:tensorflow:loss = 0.73264664, step = 2
INFO:tensorflow:Saving checkpoints for 10 into /tmp/model_dir/model.ckpt.
INFO:tensorflow:Loss for final step: 0.6661602.


In [50]:
keras_estimator.get_variable_names()

['dense_10/bias',
 'dense_10/kernel',
 'dense_11/bias',
 'dense_11/kernel',
 'global_step']